In [0]:
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
df = sns.load_dataset('diamonds')
df.sample(5)

,carat,cut,color,clarity,depth,table,price,x,y,z
3366,0.25,Very Good,G,VS2,63.0,54.0,407,4.04,4.06,2.55
13888,1.28,Very Good,I,SI2,62.6,58.0,5654,6.87,6.89,4.32
18676,1.50,Premium,H,SI1,61.4,56.0,7636,7.47,7.34,4.55
42826,0.57,Ideal,I,VS1,61.6,56.0,1355,5.33,5.35,3.29
46800,0.70,Premium,F,SI2,60.3,58.0,1806,5.79,5.76,3.48


## Groupby
---

In [6]:
df.groupby('cut').mean()

,carat,depth,table,price,x,y,z
cut,,,,,,,
Fair,1.046137,64.041677,59.053789,4358.757764,6.246894,6.182652,3.982770
Good,0.849185,62.365879,58.694639,3928.864452,5.838785,5.850744,3.639507
Ideal,0.702837,61.709401,55.951668,3457.541970,5.507451,5.520080,3.401448
Premium,0.891955,61.264673,58.746095,4584.257704,5.973887,5.944879,3.647124
Very Good,0.806381,61.818275,57.956150,3981.759891,5.740696,5.770026,3.559801


In [7]:
df.groupby('cut').median()

,carat,depth,table,price,x,y,z
cut,,,,,,,
Fair,1.00,65.0,58.0,3282.0,6.175,6.10,3.97
Good,0.82,63.4,58.0,3050.5,5.980,5.99,3.70
Ideal,0.54,61.8,56.0,1810.0,5.250,5.26,3.23
Premium,0.86,61.4,59.0,3185.0,6.110,6.06,3.72
Very Good,0.71,62.1,58.0,2648.0,5.740,5.77,3.56


In [10]:
df.groupby('cut')['carat'].count()

cut
Fair          1610
Good          4906
Ideal        21551
Premium      13791
Very Good    12082
Name: carat, dtype: int64

In [18]:
for key_group, group in df.groupby('cut'):
  grouped_price = group['price'].mean()
  print(f'Cut: {key_group}, Price: {grouped_price}')

Cut: Fair, Price: 4358.757763975155
Cut: Good, Price: 3928.864451691806
Cut: Ideal, Price: 3457.541970210199
Cut: Premium, Price: 4584.2577042999055
Cut: Very Good, Price: 3981.7598907465654


## Groupby (Multi-index)
---

In [28]:
df.groupby(['cut', 'color'])['price'].mean().to_frame().loc['Fair']

,price
color,
D,4291.061350
E,3682.312500
F,3827.003205
G,4239.254777
H,5135.683168
I,4685.445714
J,4975.655462


### Own functions with aggregate

In [32]:
df.groupby(['cut', 'color'])['price'].aggregate(['min', np.mean, max]).loc['Fair']

,min,mean,max
color,,,
D,536,4291.061350,16386
E,337,3682.312500,15584
F,496,3827.003205,17995
G,369,4239.254777,18574
H,659,5135.683168,18565
I,735,4685.445714,18242
J,416,4975.655462,18531


In [34]:
def mean_kg(x):
  return np.mean(x)/1_000
df.groupby(['cut', 'color'])['price'].aggregate(['min', np.mean, max, mean_kg]).head(5)

min         mean    max   mean_kg
cut  color                                   
Fair D      536  4291.061350  16386  4.291061
     E      337  3682.312500  15584  3.682313
     F      496  3827.003205  17995  3.827003
     G      369  4239.254777  18574  4.239255
     H      659  5135.683168  18565  5.135683

In [37]:
dict_agg = {
    'carat': [min, max],
    'price': [np.mean, mean_kg]
}
df.groupby(['cut', 'color']).aggregate(dict_agg).head(5)

carat              price          
             min   max         mean   mean_kg
cut  color                                   
Fair D      0.25  3.40  4291.061350  4.291061
     E      0.22  2.04  3682.312500  3.682313
     F      0.25  2.58  3827.003205  3.827003
     G      0.23  2.60  4239.254777  4.239255
     H      0.33  4.13  5135.683168  5.135683

### Filter

In [39]:
def f_filter(x):
  return mean_kg(x['price']) > 4

df.groupby('cut').filter(f_filter)

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,60.2,62.0,345,3.79,3.75,2.27
...,...,...,...,...,...,...,...,...,...,...
53928,0.79,Premium,E,SI2,61.4,58.0,2756,6.03,5.96,3.68
53930,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53931,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43
53934,0.72,Premium,D,SI1,62.7,59.0,2757,5.69,5.73,3.58
